# 建立 R 語言與資料庫的連結

## 郭耀仁

## 觀念

- 我們現在在做的事情是什麼？
    - 建立外部應用程式（External Applications）與資料庫的連結
- 這門課用的外部應用程式：
    - MySQL Workbench
    - R
- 這門課用的資料庫系統與主機
    - MySQL@Cloud SQL(Google Cloud Platform)

## 觀念（2）

- 不同的外部應用程式對應不同的連結方式
- 不同的主機對應不同的連結方式
- 我們用的連結方式：

|外部應用程式|連結方式|認證方式|
|----------|-------|-------|
|MySQL Workbench|IP 位址|SSL 憑證|
|R Language|代理伺服器|API 憑證|

## 觀念（3）

- Proxy: 代理伺服器
- 使用本機 127.0.0.1（or known as localhost）作為一個代理伺服器
- 進而將 R 語言與我們的 Cloud SQL 資料庫（35.189.143.131）連結
    - 像是這樣：R -> 代理伺服器 -> Cloud SQL
- 公司內的設定需要 Case by case，沒有辦法一招打通關

## 步驟

- 啟動 Cloud SQL 專案的 API 功能
- 安裝 Cloud SQL Proxy
- 建立一個 Service Account
- 下載 API 憑證
- 啟動代理伺服器
- 啟動 RStudio
- 安裝 `RMySQL` 套件
- 載入 `DBI` 套件
- 建立與代理伺服器的連線

## 步驟（2）

- 啟動 Cloud SQL 專案的 API 功能

![](img/chapter0301.png)

## 步驟（3）

- 安裝 Cloud SQL Proxy（以 OS X 64-bit 為例）

```bash
curl -o cloud_sql_proxy https://dl.google.com/cloudsql/cloud_sql_proxy.darwin.amd64
chmod +x cloud_sql_proxy
```

## 步驟（4）

- 建立一個 Service Account

![](img/chapter0302.png)

## 步驟（5）

- 下載 API 憑證

![](img/chapter0303.png)

## 步驟（6）

- 啟動代理伺服器

```bash
./cloud_sql_proxy -instances=<INSTANCE_CONNECTION_NAME>=tcp:3306 -credential_file=<PATH_TO_KEY_FILE>
```

- 以我們的例子來說，INSTANCE_CONNECTION_NAME 為 csie-r-sql:asia-northeast1:csiersql
- PATH_TO_KEY_FILE 為 ~/Downloads/csie-r-sql-4ac752c2eb6d.json

## 步驟（7）

- 啟動 RStudio
- 安裝 `RMySQL` 套件
- 載入 `DBI` 套件

```r
install.packages("RMySQL")
library(DBI)
```

## 步驟（8）

- 建立與代理伺服器的連線

```r
getSqlConnection <- function(){
    con <- dbConnect(
        RMySQL::MySQL(),
        username = 'student',
        password = 'student',
        host = '127.0.0.1', # 透過本機作為代理伺服器連線
        dbname = 'world'
    )
    return(con)
}

conn <- getSqlConnection()
res <- dbListTables(conn)
print(res)
dbDisconnect(conn) # 關閉連線
```

## 參考連結

- [Connecting mysql Client Using the Cloud SQL Proxy](https://cloud.google.com/sql/docs/mysql/connect-admin-proxy)
- [Google Cloud Platform for Data Scientists: Using R with Google Cloud SQL for MySQL](https://cloud.google.com/blog/big-data/2017/03/google-cloud-platform-for-data-scientists-using-r-with-google-cloud-sql-for-mysql)